# Explorative Datenanalyse - Energiedaten

**Ziel:** Erste Analyse der Energiedaten von SMARD (Bundesnetzagentur)

**Inhalte:**
1. Daten laden
2. Datenqualität prüfen
3. Grundlegende Visualisierungen
4. Statistische Eigenschaften
5. Saisonalität & Trends
6. Autokorrelation

In [1]:
# Standard imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Figure size
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['font.size'] = 11

print("✅ Libraries loaded successfully")

ModuleNotFoundError: No module named 'seaborn'

## 1. Daten laden

Wir nutzen die SMARD API der Bundesnetzagentur.

**Verfügbare Datenquellen:**
- `solar`: Photovoltaik-Erzeugung
- `wind_onshore`: Windkraft Onshore
- `wind_offshore`: Windkraft Offshore
- `consumption`: Stromverbrauch
- `price_day_ahead`: Day-Ahead Marktpreis
- `generation_total`: Gesamte Stromerzeugung
- Weitere siehe `src/data/smard_loader.py`

In [ ]:
from data.smard_loader import load_smard_data, SMARDDataLoader

# Konfiguration
DATA_TYPE = 'solar'  # Ändere hier für andere Datenquellen
START_DATE = '2022-01-01'
END_DATE = '2024-12-31'
RESOLUTION = 'hour'

print(f"Lade {DATA_TYPE}-Daten von {START_DATE} bis {END_DATE}...\n")

In [ ]:
# Daten laden (wird gecached für schnelleres Nachladen)
df = load_smard_data(
    filter_name=DATA_TYPE,
    start_date=START_DATE,
    end_date=END_DATE,
    resolution=RESOLUTION,
    cache_dir=Path('../data/raw')
)

print(f"\n✅ Daten erfolgreich geladen: {len(df)} Datenpunkte")

In [ ]:
# Erste Inspektion
print("Erste 5 Zeilen:")
display(df.head())

print("\nLetzte 5 Zeilen:")
display(df.tail())

print("\nDatentypen:")
print(df.dtypes)

print(f"\nZeitraum: {df['timestamp'].min()} bis {df['timestamp'].max()}")
print(f"Anzahl Tage: {(df['timestamp'].max() - df['timestamp'].min()).days}")

## 2. Datenqualität

Überprüfung auf:
- Fehlende Werte
- Duplikate
- Ausreißer
- Zeitlücken

In [ ]:
print("=" * 60)
print("DATENQUALITÄTS-REPORT")
print("=" * 60)

# Fehlende Werte
missing = df['value'].isna().sum()
missing_pct = missing / len(df) * 100
print(f"\n1. Fehlende Werte: {missing} ({missing_pct:.2f}%)")

# Duplikate
duplicates = df.duplicated(subset=['timestamp']).sum()
print(f"\n2. Duplikate (Timestamp): {duplicates}")

# Zeitlücken prüfen
df_sorted = df.sort_values('timestamp').reset_index(drop=True)
time_diffs = df_sorted['timestamp'].diff()

if RESOLUTION == 'hour':
    expected_diff = pd.Timedelta(hours=1)
elif RESOLUTION == 'day':
    expected_diff = pd.Timedelta(days=1)
else:
    expected_diff = None

if expected_diff:
    gaps = time_diffs[time_diffs > expected_diff]
    print(f"\n3. Zeitlücken (> {expected_diff}): {len(gaps)}")
    if len(gaps) > 0:
        print("   Erste 5 Lücken:")
        for idx in gaps.head().index:
            print(f"     {df_sorted.loc[idx-1, 'timestamp']} -> {df_sorted.loc[idx, 'timestamp']}")

# Statistik
print(f"\n4. Wertebereich:")
print(f"   Min: {df['value'].min():,.0f}")
print(f"   Max: {df['value'].max():,.0f}")
print(f"   Mean: {df['value'].mean():,.0f}")
print(f"   Median: {df['value'].median():,.0f}")
print(f"   Std: {df['value'].std():,.0f}")

# Nullwerte
zeros = (df['value'] == 0).sum()
zeros_pct = zeros / len(df) * 100
print(f"\n5. Nullwerte: {zeros} ({zeros_pct:.2f}%)")

# Negative Werte
negatives = (df['value'] < 0).sum()
print(f"\n6. Negative Werte: {negatives}")

print("\n" + "=" * 60)

## 3. Visualisierung - Gesamtzeitreihe

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df['timestamp'], df['value'], linewidth=0.5, alpha=0.8)
ax.set_title(f'{DATA_TYPE.upper()} - Gesamtzeitreihe', fontsize=16, fontweight='bold')
ax.set_xlabel('Datum', fontsize=12)
ax.set_ylabel('Wert [MW]', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Verteilungsanalyse

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogramm
axes[0].hist(df['value'].dropna(), bins=100, alpha=0.7, edgecolor='black')
axes[0].set_title('Verteilung der Werte', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Wert [MW]')
axes[0].set_ylabel('Häufigkeit')
axes[0].grid(True, alpha=0.3)

# Box-Plot
axes[1].boxplot(df['value'].dropna(), vert=True)
axes[1].set_title('Box-Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Wert [MW]')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistik ausgeben
print("\nStatistische Kennzahlen:")
print(df['value'].describe())

## 5. Saisonalität analysieren

### 5.1 Monatliche Muster

In [ ]:
# Monat und Jahr extrahieren
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek  # 0=Montag, 6=Sonntag
df['dayofyear'] = df['timestamp'].dt.dayofyear

# Durchschnitt pro Monat
monthly_avg = df.groupby('month')['value'].mean()

fig, ax = plt.subplots(figsize=(12, 5))
monthly_avg.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
ax.set_title('Durchschnittlicher Wert pro Monat', fontsize=14, fontweight='bold')
ax.set_xlabel('Monat')
ax.set_ylabel('Durchschnittswert [MW]')
ax.set_xticklabels(['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 
                    'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez'], rotation=0)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 5.2 Wochentags-Muster

In [ ]:
# Durchschnitt pro Wochentag
weekday_avg = df.groupby('dayofweek')['value'].mean()

fig, ax = plt.subplots(figsize=(10, 5))
weekday_avg.plot(kind='bar', ax=ax, color='coral', edgecolor='black')
ax.set_title('Durchschnittlicher Wert pro Wochentag', fontsize=14, fontweight='bold')
ax.set_xlabel('Wochentag')
ax.set_ylabel('Durchschnittswert [MW]')
ax.set_xticklabels(['Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'So'], rotation=0)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 5.3 Tages-Muster (Stundenprofil)

In [ ]:
if RESOLUTION == 'hour':
    # Durchschnitt pro Stunde
    hourly_avg = df.groupby('hour')['value'].mean()
    
    fig, ax = plt.subplots(figsize=(14, 5))
    hourly_avg.plot(ax=ax, marker='o', linewidth=2, markersize=6, color='green')
    ax.set_title('Durchschnittliches Tagesprofil (pro Stunde)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Stunde des Tages')
    ax.set_ylabel('Durchschnittswert [MW]')
    ax.set_xticks(range(24))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Stundenprofil nur für stündliche Auflösung verfügbar")

### 5.4 Jahresverlauf

In [ ]:
# Tagesdurchschnitt
daily_avg = df.groupby(df['timestamp'].dt.date)['value'].mean()

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(daily_avg.index, daily_avg.values, linewidth=1)
ax.set_title('Täglicher Durchschnittswert', fontsize=14, fontweight='bold')
ax.set_xlabel('Datum')
ax.set_ylabel('Durchschnitt [MW]')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Autokorrelation (ACF) und Partielle Autokorrelation (PACF)

Wichtig für:
- Erkennung von Abhängigkeiten
- ARIMA-Parameter-Auswahl

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Nur nicht-fehlende Werte
values_clean = df['value'].dropna()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ACF
plot_acf(values_clean, lags=168, ax=axes[0])  # 7 Tage für stündliche Daten
axes[0].set_title('Autokorrelationsfunktion (ACF)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag')

# PACF
plot_pacf(values_clean, lags=168, ax=axes[1])
axes[1].set_title('Partielle Autokorrelationsfunktion (PACF)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag')

plt.tight_layout()
plt.show()

## 7. Stationaritätstest

Augmented Dickey-Fuller Test:
- H0: Zeitreihe ist nicht-stationär
- H1: Zeitreihe ist stationär
- p-value < 0.05 → Zeitreihe ist stationär

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# ADF Test
result_adf = adfuller(values_clean)

print("=" * 60)
print("STATIONARITÄTSTEST")
print("=" * 60)
print("\nAugmented Dickey-Fuller Test:")
print(f"  ADF Statistic: {result_adf[0]:.6f}")
print(f"  p-value: {result_adf[1]:.6f}")
print(f"  Critical Values:")
for key, value in result_adf[4].items():
    print(f"    {key}: {value:.3f}")

if result_adf[1] < 0.05:
    print("\n  ✅ Zeitreihe ist STATIONÄR (p < 0.05)")
else:
    print("\n  ⚠️  Zeitreihe ist NICHT-STATIONÄR (p >= 0.05)")
    print("      → Differenzierung könnte notwendig sein")

# KPSS Test (zusätzlich)
try:
    result_kpss = kpss(values_clean, regression='ct')
    print("\n\nKPSS Test (Trend-Stationarität):")
    print(f"  KPSS Statistic: {result_kpss[0]:.6f}")
    print(f"  p-value: {result_kpss[1]:.6f}")
    
    if result_kpss[1] > 0.05:
        print("\n  ✅ Zeitreihe ist TREND-STATIONÄR (p > 0.05)")
    else:
        print("\n  ⚠️  Zeitreihe ist NICHT TREND-STATIONÄR (p <= 0.05)")
except:
    print("\nKPSS Test konnte nicht durchgeführt werden")

print("\n" + "=" * 60)

## 8. Zusammenfassung & Nächste Schritte

### Erkenntnisse aus der EDA:

**Bitte trage hier deine Beobachtungen ein:**

1. **Datenqualität:**
   - [ ] Fehlende Werte vorhanden? → Strategie festlegen
   - [ ] Ausreißer vorhanden? → Behandlung notwendig?
   - [ ] Zeitlücken? → Interpolation?

2. **Saisonalität:**
   - [ ] Tägliche Saisonalität erkennbar?
   - [ ] Wöchentliche Saisonalität?
   - [ ] Jährliche Saisonalität?

3. **Trend:**
   - [ ] Aufwärtstrend?
   - [ ] Abwärtstrend?
   - [ ] Kein Trend?

4. **Stationarität:**
   - [ ] Stationär?
   - [ ] Differenzierung notwendig?

### Nächste Schritte:

1. **Datenaufbereitung** → `02_data_preprocessing.ipynb`
   - Fehlende Werte behandeln
   - Ausreißer behandeln
   - Features engineering
   - Train/Test/Validation Split

2. **Baseline-Modelle** → `03_baseline_models.ipynb`
   - Naive Forecast
   - Seasonal Naive
   - Moving Average

3. **Statistische Modelle** → `04_statistical_models.ipynb`
   - SARIMA(X)
   - ETS

4. **Machine Learning** → `05_ml_tree_models.ipynb`
   - XGBoost, LightGBM, CatBoost

5. **Deep Learning** → `06_deep_learning_models.ipynb`
   - LSTM, GRU

6. **Advanced Models** → `07_advanced_models.ipynb`
   - TFT, N-BEATS

7. **Model Comparison** → `08_model_comparison.ipynb`

In [ ]:
# Optional: Aufbereitete Daten speichern für weitere Notebooks
output_file = Path('../data/processed') / f'{DATA_TYPE}_{START_DATE}_{END_DATE}_initial.csv'
df.to_csv(output_file, index=False)
print(f"✅ Daten gespeichert: {output_file}")